In [1]:
import tkinter as tk
import random
import time
from PIL import Image, ImageTk
import os

In [2]:
class BlackjackGame:
    def __init__(self, root):
        self.root = root
        self.root.title("Лаба 4. Blackjack")
        
        self.money = 100
        self.pot = 10
        self.score = 0

        self.canvas = tk.Canvas(self.root, width=600, height=500, bg="green")
        self.canvas.pack()

        self.button_frame = tk.Frame(self.root, width=600, height=50, bg="red")
        self.button_frame.pack()

        self.next_button = tk.Button(self.button_frame, text="Next Round", command=self.play_round, width=100, height=30)
        self.quit_button = tk.Button(self.button_frame, text="Quit", command=self.root.destroy, width=100, height=30)

        self.deck_coords_1 = (420, 10, 470, 85)
        self.deck_coords_2 = (480, 10, 530, 85)
        self.deck_final_coords = (450, 10, 500, 85)
        
        self.deck_image_1 = self.canvas.create_rectangle(self.deck_coords_1, fill="brown")
        self.deck_image_2 = self.canvas.create_rectangle(self.deck_coords_2, fill="brown")
        
        self.card_images = {}
        self.load_card_images()

        self.result_label = tk.Label(self.canvas, text="", font=("Arial", 36), bg="green", fg="black")
        self.pot_label = tk.Label(self.canvas, text="", font=("Arial", 16), bg="green", fg="black")
        self.score_label = tk.Label(self.canvas, text="", font=("Arial", 16), bg="green", fg="black")

    def load_card_images(self):
        for file_name in list(os.walk('cards'))[0][2]:
            image = Image.open(f"cards/{file_name}")
            keys = file_name.split('_')
            value, suit = keys[0], keys[1][:-4]
            image = image.resize((50, 75), Image.Resampling.LANCZOS)
            self.card_images[(value, suit)] = ImageTk.PhotoImage(image)

    def shuffle_deck_animation(self):
        for _ in range(6):
            self.canvas.move(self.deck_image_1, 5, 0)
            self.canvas.move(self.deck_image_2, -5, 0)           
            self.root.update()
            time.sleep(0.05)

        self.canvas.coords(self.deck_image_1, self.deck_final_coords)
        self.canvas.coords(self.deck_image_2, self.deck_final_coords)
    
    def animate_card(self, x, y, card):
        for i in range(15):
            self.canvas.move(self.deck_image_2, (x - 450) / 15, (y - 10) / 15)
            self.root.update()
            time.sleep(0.05)

        self.canvas.coords(self.deck_image_2, self.deck_final_coords)

        card_image = self.card_images[card]
        self.canvas.create_image(x, y, image=card_image, anchor=tk.NW, tags="cards")

    def play_round(self):
        self.canvas.delete("cards")
        self.result_label.place_forget()
        self.pot_label.place_forget()
        self.score_label.place_forget()
        self.next_button.place_forget()
        self.quit_button.place_forget()

        self.canvas.coords(self.deck_image_1, self.deck_coords_1)
        self.canvas.coords(self.deck_image_2, self.deck_coords_2)
        
        self.shuffle_deck_animation()

        round_cards = random.sample(list(self.card_images.keys()), 4)
        player_cards = round_cards[:2]
        dealer_cards = round_cards[2:]
        
        player_score = sum(int(card[0]) if card[0] not in ['jack', 'king', 'queen'] else 10 for card in player_cards)
        dealer_score = sum(int(card[0]) if card[0] not in ['jack', 'king', 'queen'] else 10 for card in dealer_cards)

        self.animate_card(50, 150, player_cards[0])
        self.animate_card(120, 150, player_cards[1])

        self.animate_card(50, 50, dealer_cards[0])
        self.animate_card(120, 50, dealer_cards[1])

        if player_score > 21:
            self.result_label.config(text="YOU BUST!")
            self.money -= self.pot
        elif dealer_score > 21 or player_score > dealer_score:
            self.result_label.config(text="YOU WIN!")
            self.money += self.pot
        else:
            self.result_label.config(text="YOU LOSE!")
            self.money -= self.pot

        self.result_label.place(x=190, y=220)
        self.pot_label.config(text=f"Pot: ${self.pot}")
        self.pot_label.place(x=320, y=150)
        self.score_label.config(text=f"Score: {player_score} Money: ${self.money}")
        self.score_label.place(x=190, y=450)
        self.next_button.place(x=150, y=10, width=100, height=30)
        self.quit_button.place(x=350, y=10, width=100, height=30)

root = tk.Tk()
game = BlackjackGame(root)
game.play_round()
root.mainloop()